<a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/deeplearning.ai/tf/c4_w2_time_series_prediction_linear_regresion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Time Series Prediction - NN Single Layer (linear regresion)

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.3.0


In [12]:
def plot_series(time, series, format='-', start=0, end=None):
  plt.plot(time[start:end], series[start:end], format)
  plt.xlabel('Time')
  plt.ylabel('Value')
  plt.grid(True)

def trend(time, slope=0):
  return slope * time

def seasonal_pattern(season_time):
  '''
  Just an arbitrary patten, you can change it if you wish
  '''
  return np.where(
      season_time < 0.4,
      np.cos(season_time * 2 * np.pi),
      1 / np.exp(3 * season_time)
  )

def seasonality(time, period, amplitude=1, phase=0):
  '''
  Repeats the same pattern at each period
  '''
  season_time = ((time + phase) % period) / period
  return amplitude * seasonal_pattern(season_time)

def noise(time, noise_level=1, seed=None):
  rnd = np.random.RandomState(seed)
  return rnd.randn(len(time)) * noise_level

In [14]:
baseline = 10
slope = 0.05
amplitude = 40
noise_level = 5
time = np.arange(4 * 365 + 1, dtype='float32')
trend_series = trend(time, slope)
seasonality_series = seasonality(time, period=365, amplitude=amplitude)
noise_series = noise(time, noise_level, seed=42)

series = baseline + trend_series + seasonality_series + noise_series

In [15]:
split_time = 1000
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

In [16]:
window_size = 20
batch_size = 32
shuffle_buffer_size = 1000